# NWS Warning Polygons

In [1]:
#Import the necessary packages
import numpy as np
import datetime as dt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import matplotlib.patches as mpatches
import matplotlib.pyplot as plt
import noaa_sdk as noaa_sdk
import pandas as pd
import shapely as shapely
from   metpy.plots       import  USCOUNTIES



import timezonefinder    as tzf
import pytz              as pytz


import geopandas as gp

In [2]:
proj_data_text = '+proj=aea +lat_1=29.5 +lat_2=45.5 +lat_0=37.5 +lon_0=-96 +x_0=0 +y_0=0 +ellps=GRS80 +datum=NAD83 +units=m +no_defs'

gif_file_name = "./graphics_files/NWS_Warnings.png"
xls_file_name = "./graphics_files/NWS_Warnings.xlsx"



myproj = ccrs.AlbersEqualArea(central_longitude=-96, 
                              central_latitude=37.5, 
                              false_easting=0.0, 
                              false_northing=0.0, 
                              standard_parallels=(29.5, 45.5))



tz='America/Denver'

time_utc = dt.datetime.utcnow()
valid_time = pd.to_datetime(time_utc).tz_localize(tz="UTC").strftime("%Y-%m-%d %H%M %Z")
local_time = pd.to_datetime(time_utc).tz_localize(tz="UTC").tz_convert(tz=tz).strftime("%Y-%m-%d %H%M %Z")

print(valid_time)
print(local_time)

2022-09-30 0440 UTC
2022-09-29 2240 MDT


In [3]:
# Open the NWS API in python to get the active alerts
n = noaa_sdk.noaa.NOAA()
alerts = n.active_alerts()


In [4]:
#injest //shapefiles/CONUS_UGC_Zones/


UGC_Zones_Shapefile    = gp.read_file('./shapefiles/CONUS_UGC_Zones/CONUS_UGC_Zones.shp')
UGC_Counties_Shapefile = gp.read_file('./shapefiles/CONUS_UGC_Counties/CONUS_UGC_Counties.shp')
 
UGC_Shapefile = pd.concat([UGC_Zones_Shapefile, 
                           UGC_Counties_Shapefile]).drop(["UGC_Temp",
                                                          "Shape_Leng",
                                                          "Shape_Le_1",
                                                          "Shape_Area",
                                                          "STATE",
                                                          "CWA",
                                                          "COUNTYNAME",
                                                          "FIPS",
                                                          "TIME_ZONE",
                                                          "FE_AREA"],
                                                          axis = 'columns').reset_index(drop=True)  
    
UGC_Zone_County_List = UGC_Shapefile['UGC'].to_list()



In [5]:
# priority warnings table
warning_priority_table = pd.read_csv("./warning_table_sorted.csv")

warning_priority_table = warning_priority_table.rename(columns={"hdln": "event"})



In [6]:

current_warnings = pd.DataFrame(columns = ['event',
                                           'UGC',
                                           'coverage',
                                           'message'])



i = 0



for alert in alerts['features']:
    event = alert['properties']['event']
    if (event != "Test Message"): 
        
        try:

            ugc_codes = alert['properties']['geocode']['UGC']
            message   = alert['properties']['@id']

            for ugc_code in ugc_codes:
                if ugc_code in UGC_Zone_County_List: 
                    if (ugc_code[2] == 'C'):
                        coverage = "County"
                    else:
                        coverage = "Zones"

                    deleteme = pd.DataFrame([[event,
                              ugc_code,
                              coverage,
                              message]], 
                              columns = ['event',
                                         'UGC',
                                         'coverage',
                                         'message'])
                    current_warnings = pd.concat([current_warnings, 
                                                  deleteme]) 
                    i = i + 1
        except KeyError:
            print("poopie:")
            print(event, ugc_code)
    
current_warnings = current_warnings.merge(warning_priority_table, how='left', on='event')
current_warnings = UGC_Shapefile.merge(current_warnings, how='right', on='UGC')
current_warnings = current_warnings.sort_values("Rank", ascending=False)


if (current_warnings['color'].isnull().values.any()):
    print("Missing Colors Found")
    locs_missing = current_warnings.index[current_warnings['color'].isnull()].tolist()
    print(current_warnings.loc[locs_missing])

  
    for loc_missing in locs_missing:
        current_warnings.loc[loc_missing,"color"] = 'red'
    
print("replaced")


warning_color_table = current_warnings[["event","color"]].drop_duplicates()


current_warnings.drop(["geometry"], axis="columns").to_excel(xls_file_name)

print("done: ",i,"rows; ",len(warning_color_table),"event types")
print()

Missing Colors Found
          LON      LAT Class     UGC WFO_CWA   Zone_Name  \
101  -83.4021  34.8818  Land  GAZ010     GSP       Rabun   
102  -83.5312  34.6310  Land  GAZ017     GSP   Habersham   
103  -83.2935  34.5540  Land  GAZ018     GSP    Stephens   
104  -83.2292  34.3755  Land  GAZ026     GSP    Franklin   
105  -82.9640  34.3511  Land  GAZ028     GSP        Hart   
...       ...      ...   ...     ...     ...         ...   
1601 -79.7480  35.0059  Land  NCZ084     RAH    Richmond   
1602 -79.4805  34.8411  Land  NCZ085     RAH    Scotland   
1603 -79.2371  35.0174  Land  NCZ086     RAH        Hoke   
1604 -78.8273  35.0485  Land  NCZ088     RAH  Cumberland   
1605 -78.3712  34.9915  Land  NCZ089     RAH     Sampson   

                                               geometry  \
101   POLYGON ((-83.10799 35.00041, -83.10629 34.999...   
102   POLYGON ((-83.59440 34.82321, -83.59219 34.821...   
103   POLYGON ((-83.33829 34.68031, -83.33209 34.681...   
104   POLYGON ((-83.09

In [7]:
# check for missing colors.

if (warning_color_table['color'].isnull().values.any()):
    print("Missing Colors Found")
    loc_missing = warning_color_table.index[warning_color_table['color'].isnull()].tolist()
    print(loc_missing)
    print(warning_color_table.loc[loc_missing])
    warning_color_table.loc[loc_missing,"color"] = '#ffffff'
    
print("replaced")
print(warning_color_table)

replaced
                           event             color
1608     Beach Hazards Statement         turquoise
1287     Coastal Flood Statement         olivedrab
1909       Rip Current Statement              aqua
1637           Air Quality Alert              gray
295       Coastal Flood Advisory         lawngreen
1329              Frost Advisory    cornflowerblue
1965        Small Craft Advisory           thistle
1620          High Surf Advisory      mediumorchid
1067               Wind Advisory               tan
1749         Coastal Flood Watch  mediumaquamarine
1897                  Gale Watch              pink
338                  Flood Watch          seagreen
1848           Flash Flood Watch         limegreen
718         Tropical Storm Watch           #f08080
641            Storm Surge Watch           #DB7FF7
1739              Freeze Warning              cyan
29                  Gale Warning           #dda0dd
231        Coastal Flood Warning       forestgreen
1186               Flo

In [8]:
legend_color_table = warning_color_table.values.tolist()
print(legend_color_table)
legend_color_table = []

for row in warning_color_table.iterrows():
    mypatch = [mpatches.Patch(color=row[1][1], label=row[1][0])]
    legend_color_table = legend_color_table + mypatch
    
    
  

[['Beach Hazards Statement', 'turquoise'], ['Coastal Flood Statement', 'olivedrab'], ['Rip Current Statement', 'aqua'], ['Air Quality Alert', 'gray'], ['Coastal Flood Advisory', 'lawngreen'], ['Frost Advisory', 'cornflowerblue'], ['Small Craft Advisory', 'thistle'], ['High Surf Advisory', 'mediumorchid'], ['Wind Advisory', 'tan'], ['Coastal Flood Watch', 'mediumaquamarine'], ['Gale Watch', 'pink'], ['Flood Watch', 'seagreen'], ['Flash Flood Watch', 'limegreen'], ['Tropical Storm Watch', '#f08080'], ['Storm Surge Watch', '#DB7FF7'], ['Freeze Warning', 'cyan'], ['Gale Warning', '#dda0dd'], ['Coastal Flood Warning', 'forestgreen'], ['Flood Warning', 'green'], ['Storm Warning', 'darkorchid'], ['Storm Surge Warning', '#B524F7'], ['Hurricane Watch', '#ff00ff'], ['Hurricane Warning', '#dc143c'], ['Tropical Storm Warning', '#b22222'], ['Tropical Cyclone Statement', 'red']]


In [9]:
print(warning_color_table)


                           event             color
1608     Beach Hazards Statement         turquoise
1287     Coastal Flood Statement         olivedrab
1909       Rip Current Statement              aqua
1637           Air Quality Alert              gray
295       Coastal Flood Advisory         lawngreen
1329              Frost Advisory    cornflowerblue
1965        Small Craft Advisory           thistle
1620          High Surf Advisory      mediumorchid
1067               Wind Advisory               tan
1749         Coastal Flood Watch  mediumaquamarine
1897                  Gale Watch              pink
338                  Flood Watch          seagreen
1848           Flash Flood Watch         limegreen
718         Tropical Storm Watch           #f08080
641            Storm Surge Watch           #DB7FF7
1739              Freeze Warning              cyan
29                  Gale Warning           #dda0dd
231        Coastal Flood Warning       forestgreen
1186               Flood Warnin

In [10]:
bbox=[-120,-73,22.5,50]

xxx = [-120.0, -73.0]
yyy = [  22.5,  50.0]


fig = plt.figure(figsize   = (11, 6), 
                 facecolor = 'white')

ax = fig.add_subplot(1, 1, 1, 
                     projection=myproj)

plt.suptitle("NWS Watches and Warnings",
             fontsize = 20, 
             color    = "black")
ax.set_extent(bbox)
ax.set_title(valid_time + "  (" + local_time+")",
             fontsize=15, 
             color="black")

current_warnings.plot(ax = ax,aspect='equal',
                      facecolor=current_warnings["color"],
                      transform=ccrs.PlateCarree(),
                      edgecolor='None', linewidth=0)

ax.add_feature(cfeature.COASTLINE.with_scale('50m'), 
               linewidth = 0.5)
ax.add_feature(cfeature.STATES.with_scale('50m'),    
               linewidth = 0.25, 
               edgecolor = 'black')
ax.add_feature(cfeature.LAKES.with_scale('50m'),   
               linewidth = 0.5,
               facecolor = "none", 
               edgecolor = 'black')
ax.add_feature(feature    = USCOUNTIES, 
                   linewidths = 0.1,
                   edgecolor  = 'black',
                   facecolor  = 'none')

ax.set_frame_on(False)

plt.subplots_adjust(left   = 0.01, 
                    right  = 0.75, 
                    top    = 0.91, 
                    bottom = 0.01)

labelspacing = 0.1
fig.legend(handles  = legend_color_table, 
           loc      = 'right',
           frameon  = False,
           labelspacing = labelspacing)




#########################################
#
# Insert a Clock
#

axins = fig.add_axes(rect     =    [0, # 0.015,
                                    1-0.12, #0.015,
                                    0.12*8/9,
                                    0.12],
                      projection  =  "polar")

time_for_clock = pd.to_datetime(time_utc).tz_localize(tz="UTC").tz_convert(tz=tz).time()

hour   = time_for_clock.hour
minute = time_for_clock.minute
second = time_for_clock.second

circle_theta  = np.deg2rad(np.arange(0,360,0.01))
circle_radius = circle_theta * 0 + 1

if (hour > 12) :
    hour = hour - 12

angles_h = 2*np.pi*hour/12+2*np.pi*minute/(12*60)+2*second/(12*60*60)
angles_m = 2*np.pi*minute/60+2*np.pi*second/(60*60)

print(time_for_clock)
print(hour,   np.rad2deg(angles_h))
print(minute, np.rad2deg(angles_m))


plt.setp(axins.get_yticklabels(), visible=False)
plt.setp(axins.get_xticklabels(), visible=False)
axins.spines['polar'].set_visible(False)
axins.set_ylim(0,1)
axins.set_theta_zero_location('N')
axins.set_theta_direction(-1)
axins.set_facecolor("white")
axins.grid(False)

axins.plot([angles_h,angles_h], [0,0.6], color="black", linewidth=1.5)
axins.plot([angles_m,angles_m], [0,0.95], color="black", linewidth=1.5)
axins.plot(circle_theta, circle_radius, color="darkgrey", linewidth=1)


#
#########################################
        
        
plt.savefig(gif_file_name,
                        facecolor   = 'white', 
                        transparent =   False)


plt.close()


print("done")

22:40:48.456248
10 320.1273239544735
40 244.79999999999998
done
